In [1]:
import pandas as pd
import numpy as np
import os
import sys

In [2]:
print(sys.maxsize)


9223372036854775807


In [3]:
master_path = 'C:/DataCleanup/Data/'
master_file = 'Data Cleanup Tracking Sheet.xlsx'
master_sheet = 'Full Tracking List'

file_df = pd.read_excel(master_path + master_file, sheet_name=master_sheet, dtype={'ProfileID_M':str, 'EID_AM': str, 
                                                                                   'Phone_M':str, 'Phone_AM': str, },
                       parse_dates={'DOB_M', 'Created_M', 'Modified_M', 'Completed'})
#file_df = pd.read_csv(master_path + master_file, low_memory=False, index_col='ProfileID_M')

file_df

OverflowError: Python int too large to convert to C int

In [ ]:
# Load list of deletes for not interested responses

In [ ]:
#not_int_df = pd.read_excel(master_path + "am_Nos.xlsx")
#not_int_df
# DONE

<H1>Create New Tracking list without all the rows at the end

In [ ]:
file_df.dropna(subset=['FirstName_M', 'LastName_M'], inplace=True)

In [ ]:
not_int_df = not_int_df.assign(result=not_int_df['email'].isin(file_df['email_M']))

In [ ]:
# Mark anyone in file_df as delete if in the above list
file_df = file_df.assign(result=file_df['email_M'].isin(not_int_df['email']))
file_df = file_df.assign(result=file_df['ProfileID_M'].isin(not_int_df['Profile ID in Mobilize']))

In [ ]:
file_df['AM Delete'] = file_df['result']
mask = file_df['AM Delete'] == True
file_df['KeeporDelete'][mask] = "Delete"
file_df['DecisionBy:'][mask] = 'Julie H.'
file_df['Removed_AM'][mask] = 'Yes'
file_df['AM_By:'][mask] = 'Connor'
file_df['Removed_Mob'][mask] = 'Yes'
file_df['MobilizeBy:'][mask] = 'Julie H.'
file_df['Completed'][mask] = '10/09/2021'

file_df.drop(columns=['result'], inplace=True)

In [ ]:
file_df

In [ ]:
not_int_df
not_int_df.to_excel(master_path + "Not Interested Not in Mobilize.xlsx")

<H1>Creating list of invalid contact info

In [ ]:
mask_000 = file_df.loc[:,'Phone_M'].str.contains('000-000')
mask_111 = file_df.loc[:,'Phone_M'].str.contains('111-111')
mask_123 = file_df.loc[:,'Phone_M'].str.contains('123-123')
mask_999 = file_df.loc[:,'Phone_M'].str.contains('999-999')
mask_888 = file_df.loc[:,'Phone_M'].str.contains('888-888')
mask_noemail = file_df.loc[:,'email_M'].str.contains('noemail')
mask_fav = file_df.loc[:,'email_M'].str.contains('favoritestaf')
mask_invalid = file_df.loc[:,'email_M'].str.contains('invalid')
mask_dontcall = file_df.loc[:,'email_M'].str.contains('dontcall')
mask_0000 = file_df.loc[:,'email_M'].str.contains('0000')
mask_donot = file_df.loc[:,'email_M'].str.contains('donot')
mask_krucial = file_df.loc[:,'email_M'].str.contains('krucialstaf')
mask_1111 = file_df.loc[:,'email_M'].str.contains('1111')

In [ ]:
bad_contact_df = file_df.loc[mask_000 | mask_111 | mask_123 | mask_999 | mask_888 | mask_noemail | mask_fav | mask_invalid | mask_dontcall |
           mask_0000 | mask_donot | mask_krucial | mask_1111, :]

In [ ]:
bad_contact_df

In [ ]:
bad_contact_df.to_excel(master_path + "bad_contact.xlsx")

In [ ]:
file_df = file_df.assign(result=file_df['ProfileID_M'].isin(bad_contact_df['ProfileID_M']))
file_df['Bad contact info'] = file_df['result']
file_df.drop(columns=['result'], inplace=True)

In [ ]:
# Make sure to not worry about terminated employees
term_df = pd.read_csv(master_path + "EmployeeListingReport.csv")

In [ ]:
file_df.drop(file_df[file_df['AM Delete']=="TRUE"])

In [ ]:
file_df = file_df.assign(result=file_df['ProfileID_M'].isin(term_df[['profileid']]))
file_df['Termed'] = file_df['result']
file_df.drop(columns=['result'], inplace=True)

In [ ]:
file_df.to_excel(master_path + "Master Tracking Sheet - Revised.xlsx")